In [0]:
!pip install eli5

     |████████████████████████████████| 112kB 3.1MB/s 


In [0]:
import pandas as pd
import numpy as np

from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor

from sklearn.metrics import mean_absolute_error
from sklearn.model_selection import cross_val_score

import eli5
from eli5.sklearn import PermutationImportance


from ast import literal_eval
from tqdm import  tqdm_notebook


/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.metrics.scorer module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.metrics. Anything that cannot be imported from sklearn.metrics is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.6/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.feature_selection.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.feature_selection. Anything that cannot be imported from sklearn.feature_selection is now part of the private API.
  warnings.warn(message, FutureWarning)
Using TensorFlow backend.


In [0]:
cd drive/My Drive/Colab Notebooks/DataWorkshops_Project_Matrix

/content/drive/My Drive/Colab Notebooks/DataWorkshops_Project_Matrix


In [0]:
df = pd.read_csv('data/men_shoes.csv')

/usr/local/lib/python3.6/dist-packages/IPython/core/interactiveshell.py:2718: DtypeWarning: Columns (29) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [0]:
df.shape

(18280, 48)

In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension', 'ean', 'features',
       'flavors', 'imageurls', 'isbn', 'keys', 'manufacturer',
       'manufacturernumber', 'merchants', 'name', 'prices_amountmin',
       'prices_amountmax', 'prices_availability', 'prices_color',
       'prices_condition', 'prices_count', 'prices_currency',
       'prices_dateadded', 'prices_dateseen', 'prices_flavor', 'prices_issale',
       'prices_merchant', 'prices_offer', 'prices_returnpolicy',
       'prices_shipping', 'prices_size', 'prices_source', 'prices_sourceurls',
       'prices_warranty', 'quantities', 'reviews', 'sizes', 'skus',
       'sourceurls', 'upc', 'vin', 'websiteids', 'weight'],
      dtype='object')

In [0]:
#the aim of analysis - predict rices_amountmin columns

mean_price = np.mean(df['prices_amountmin'])
mean_price

93.53952188183217

In [0]:
#true value
y_true = df['prices_amountmin']

#predicted value as mean
y_pred = [mean_price] * y_true.shape[0]

In [0]:
mean_absolute_error(y_true, y_pred)

63.968341227870624

In [0]:
#true value
y_true = df['prices_amountmin']

#predicted value as mean
y_pred = [np.median(y_true)] * y_true.shape[0]
mean_absolute_error(y_true, y_pred)

58.01120240700219

In [0]:
#use log to change long tail (+1 to avoid log(0) error)
price_log_mean = np.expm1(np.mean(np.log(y_true + 1)))
price_log_mean


#true value
y_true = df['prices_amountmin']

#predicted value as mean
y_pred = [price_log_mean] * y_true.shape[0]
mean_absolute_error(y_true, y_pred)

58.03539547661041

In [0]:
#decision tree with brands columns
df['brand'].value_counts()

Nike                                  1709
PUMA                                   574
Ralph Lauren                           524
New Balance                            361
VANS                                   356
                                      ... 
Alpha Tactical                           1
Best Choice Products                     1
Wood N Stream                            1
Signature by Levi Strauss &amp; Co       1
GLOBE                                    1
Name: brand, Length: 1856, dtype: int64

In [0]:
#set unique id to string (retun id to string)
df['brand_cat'] = df['brand'].factorize()[0]
df['manufacturer_cat'] = df['manufacturer'].factorize()[0]

In [0]:
#function to return model based on features
# X- matrix of features
features = ['manufacturer_cat']


def run_model(features, model = DecisionTreeRegressor(max_depth=5)):
  X = df[features].values
  y = df['prices_amountmin'].values

  

  scores = cross_val_score(model, X, y, scoring = 'neg_mean_absolute_error')

  return np.mean(scores), np.std(scores)


In [0]:
run_model(['brand_cat'])

(-58.38655694633361, 4.223555478221712)

In [0]:
model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model )

(-57.47223572384038, 4.328288468270897)

In [0]:
#remove case sensitive names in brand

df['brand_cat'] = df['brand'].map(lambda x: str(x).lower()).factorize()[0]

model = RandomForestRegressor(max_depth=5, n_estimators=100, random_state=0)
run_model(['brand_cat'], model )

(-57.31783843165656, 4.181246596160967)

In [0]:
#parser json column

def parse_features(x):
  output_dic = {}
  if str(x) == 'nan': return output_dic

  features = literal_eval(x.replace('\\"','"'))

  for item in features:
    key = item['key'].lower().strip()
    value = item['value'][0].lower().strip()

    output_dic[key] = value
    
  return output_dic


df['features_parsed'] = df['features'].map(parse_features)
df['features_parsed'].head()

0    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
1    {'gender': 'men', 'shoe size': 'm', 'shoe cate...
2    {'gender': 'men', 'color': 'black', 'shipping ...
3    {'gender': 'men', 'color': 'black', 'shipping ...
4    {'gender': 'men', 'color': 'black', 'shipping ...
Name: features_parsed, dtype: object

In [0]:
#what kind of features exist

keys = set()

#to get only unique value
df['features_parsed'].map(lambda x: keys.update(x.keys()))

len(keys)

476

In [0]:
#new column for each unique value in keys

# function to avoid mistake in nameing new columns
def get_name_feat(key):
  return 'feat_' + key

for key in tqdm_notebook(keys):
  #map function checking if keys is in feates
  df['features_parsed'].map(lambda feats: feats[key] if key in feats else np.nan)




In [0]:
df.columns

Index(['id', 'asins', 'brand', 'categories', 'colors', 'count', 'dateadded',
       'dateupdated', 'descriptions', 'dimension',
       ...
       'feat_lined', 'feat_golf shoe type', 'feat_sku', 'feat_light weight',
       'feat_clothing product type', 'feat_impact resistant',
       'feat_recommended use', 'feat_has expiration', 'feat_inseam',
       'feat_hood'],
      dtype='object', length=527)

In [0]:
#check procentage of nulls by clumns, some coulmn culd be useless

keys_stats = {}

for key in keys:
  keys_stats[key] = df[False == df[get_name_feat(key)].isnull()].shape[0] / df.shape[0] * 100

keys_stats

{'100% adidas authentic': 0.005470459518599562,
 'accessory type': 0.1422319474835886,
 'adidas': 0.005470459518599562,
 'adjustable': 0.34463894967177244,
 'age': 0.6400437636761488,
 'age end': 0.4431072210065646,
 'age gender group': 0.12582056892778995,
 'age group': 27.64770240700219,
 'age range': 0.5470459518599562,
 'age segment': 0.1422319474835886,
 'age start': 0.4431072210065646,
 'airport friendly': 0.005470459518599562,
 'alarm': 0.23522975929978115,
 'amazonbestsellersrank': 0.06017505470459519,
 'animal type': 0.005470459518599562,
 'antiscratch lens coating': 0.005470459518599562,
 'applicable': 0.02188183807439825,
 'arm': 0.6236323851203501,
 'article': 0.005470459518599562,
 'assembled in country of origin': 2.199124726477024,
 'assembled product dimensions (l x w x h)': 10.300875273522976,
 'assembled product weight': 1.5536105032822756,
 'athlete': 0.0437636761487965,
 'atpv arc rating': 0.005470459518599562,
 'attachment': 0.03282275711159737,
 'audience': 0.2352

In [40]:
#filter only important columns

{k:v for k,v in keys_stats.items() if v > 30}

{'brand': 48.62691466083151,
 'color': 47.784463894967175,
 'gender': 50.17505470459519,
 'manufacturer part number': 36.252735229759296,
 'material': 34.9070021881838}

In [0]:
#factorize the most important features 

df['feat_brand_cat'] = df['feat_brand'].factorize()[0]
df['feat_color_cat'] = df['feat_color'].factorize()[0]
df['feat_manufacturer part number_cat'] = df['feat_manufacturer part number'].factorize()[0]
df['feat_material_cat'] = df['feat_material'].factorize()[0]



In [43]:
#model only old columns
model = RandomForestRegressor(max_depth=5, n_estimators=5)
run_model(['brand_cat'], model)

(-57.5190798320653, 4.138236158760471)

In [50]:
#model on new columns
feats = ['feat_brand_cat','feat_color_cat','feat_manufacturer part number_cat','feat_material_cat']

run_model(feats, model)

(-60.96868929060925, 3.949949055334124)

In [52]:
model = RandomForestRegressor(max_depth=5, n_estimators=5, random_state=0)

X = df[feats].values
y = df['prices_amountmin'].values

model.fit(X,y)

#to print only important features

perm = PermutationImportance(model, random_state= 1).fit(X,y)

eli5.show_weights(perm, feature_names = feats)

Weight,Feature
0.0995 ± 0.0049,feat_material_cat
0.0954 ± 0.0021,feat_manufacturer part number_cat
0.0820 ± 0.0055,feat_brand_cat
0.0601 ± 0.0028,feat_color_cat


In [55]:
#model on new columns
feats = ['feat_brand_cat','feat_manufacturer part number_cat','feat_material_cat']

run_model(feats, model)

(-60.91300165650205, 3.8570647964083373)

In [57]:
df['brand'].value_counts(normalize = True)

Nike                                  0.093577
PUMA                                  0.031430
Ralph Lauren                          0.028692
New Balance                           0.019767
VANS                                  0.019493
                                        ...   
Alpha Tactical                        0.000055
Best Choice Products                  0.000055
Wood N Stream                         0.000055
Signature by Levi Strauss &amp; Co    0.000055
GLOBE                                 0.000055
Name: brand, Length: 1856, dtype: float64

In [60]:
df[df['brand'] == 'Nike'].features_parsed.head().values

array([{'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'sport': 'soccer', 'main color': 'orange', 'type': 'cleats', 'condition': 'new without box'},
       {'style': 'athletic sneakers', 'condition': 'new with box'}, {}],
      dtype=object)

In [0]:
!git add Day_5.ipynb
!git config --global user.email "arkadiusz.wos@gmail.com"
!git config --global user.name "ArkadiuszWos"

!git commit -m "first model prediction"
!git push

[master 2502f72] first model prediction
 1 file changed, 1 insertion(+)
 create mode 100644 Day_4.ipynb
Counting objects: 3, done.
Delta compression using up to 2 threads.
Compressing objects: 100% (3/3), done.
Writing objects: 100% (3/3), 3.47 KiB | 1.16 MiB/s, done.
Total 3 (delta 1), reused 0 (delta 0)
remote: Resolving deltas: 100% (1/1), completed with 1 local object.
To https://github.com/ArkadiuszWos/DataWorkshops_Project_Matrix.git
   166ae4e..2502f72  master -> master
